# Apply the trained model

Take a batch file of sample points where the GSV images have been cached, and apply the model to detect the required objects and log where they were found.

## Configuration

In [1]:
split_subdir         = 'dashcam_tour_mount_eliza'
#split_subdir         = 'dashcam_tour_frankston'

margin               = 0
confidence_threshold = 0.60

#mask = None
mask = [[0, 0], [0, 720], [480, 650], [950, 650], [950, 0]]

# Filters for outlier hits
min_hits_in_range    = 2
min_range_from_hit   = 10
max_range_from_hit   = 50

#trained_model_name   = 'centernet_hg104_512x512_coco17_tpu-8'
#trained_model_name   = 'centernet_V1' # Trained on GSV only
#trained_model_name   = 'centernet_V2' # Trained on GSV plus some dashcam footage
#trained_model_name   = 'centernet_V3' # Trained on CSV plus dashcam with false positives (2000 steps)
#trained_model_name   = 'centernet_V4' # Add RoadDefect and RoadWriting classes to model (2000 steps)
trained_model_name   = 'centernet_V5' # V4 but with 30,000 steps

## Load and configure model

In [2]:
import os
import sys
from pathlib import Path

import numpy as np

from ipyleaflet import Map, GeoJSON


# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import tf2_utils.tf2_model_wrapper as tf2_model_wrapper
import osm_gsv_utils.detection_map as detection_map
import osm_gsv_utils.detection_log_filter as detection_log_filter

%matplotlib inline

In [3]:
# Derive paths

download_directory = os.path.join(module_path_root, 'data_sources', split_subdir, 'split')
batch_filename     = os.path.join(module_path_root, 'data_sources', split_subdir, 'split', 'metadata.csv')
output_directory   = os.path.join(module_path_root, 'data_sources', split_subdir, 'detections')

os.chdir(Path(module_path_root).parent.absolute())

# Initialise numpy version of mask
if mask is not None:
    mask_np = np.array(mask)
else:
    mask_np = None

In [ ]:
# Initialise model
model_wrapper = tf2_model_wrapper(split_subdir, margin, download_directory, output_directory, trained_model_name)

## Model Application

In [ ]:
# Run detection for entire batch
detection_log = model_wrapper.process_split_dir(
    batch_filename,
    min_score = confidence_threshold,
    mask      = mask_np,
    progress  = True,
    verbose   = False
)

In [ ]:
# Example of bike lane FILE210926-101400F_0600.png
# Example of give way FILE210926-101400F_0996.png

#model_wrapper.apply_model(0, 0, 0, 0, 0, 0, 0, filename='FILE210926-101400F_0600.png', heading_offsets=[0], mask=mask_np, display=True)

In [ ]:
#model_wrapper.apply_model(0, 0, 0, 0, 0, 0, 0, filename='FILE210926-101400F_0996.png', heading_offsets=[0], mask=mask_np, display=True)

## Filter hits to remove outliers

In [4]:
filter = detection_log_filter(os.path.join(output_directory, 'detection_log.csv'))

filter.apply_filter(
    os.path.join(output_directory, 'detection_log_filtered.csv'),
    min_hits_in_range  = 2,
    min_range_from_hit = 5,
    max_range_from_hit = 200
)

  0%|          | 0/526 [00:00<?, ?it/s]